# Prototype data extraction pipeline

tl;dr;
- for easy stuff, like location, sites, year, they are quite reliable
- The model seems to be able to read some information from the table, but the accuracy needs to be improved with more tuning.
- We will start with extracting information/data from the text for now, and then develop an improved version for extracting the data from the figures.

Note:
- Structured output is not supported in Assistant API, so we will not export it to PDF for now.

In [9]:
from pathlib import Path

import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel

from proto_carbongpt.data_model import Paper

load_dotenv()


True

In [4]:
df = pd.read_csv("data/Template_output.csv")

In [5]:
# PDF files
pdfs_dir = Path("data/pdfs")
pdfs = list(pdfs_dir.glob("*.pdf"))
pdf = pdfs[0]
print(pdf)


data/pdfs/Villa et al., 2024.pdf


In [7]:
client = OpenAI()

# Only upload once
# uploaded_file = client.files.create(file=open(pdf, "rb"), purpose="assistants")
# file_id = uploaded_file["id"]
print(client.files.list())
file_id = "file-CDPqNwobM5psUr5kk1woEVUu"

SyncPage[FileObject](data=[FileObject(id='file-CDPqNwobM5psUr5kk1woEVUu', bytes=1560028, created_at=1728059232, filename='Villa et al., 2024.pdf', object='file', purpose='assistants', status='processed', status_details=None)], object='list', has_more=False, first_id='file-CDPqNwobM5psUr5kk1woEVUu', last_id='file-CDPqNwobM5psUr5kk1woEVUu')


In [8]:
client.beta.assistants.list()


SyncCursorPage[Assistant](data=[Assistant(id='asst_ruWqBYrKwmsNT2igLfaZnjFR', created_at=1728061335, description=None, instructions='You are a research assistant. Please extract the following information from the provided PDF.', metadata={}, model='gpt-4o', name=None, object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_MR92QgH4Xx0fpkr249Wfbo9g'])), top_p=1.0)], object='list', first_id='asst_ruWqBYrKwmsNT2igLfaZnjFR', last_id='asst_ruWqBYrKwmsNT2igLfaZnjFR', has_more=False)

In [35]:
instructions = f"""
Please extract the following information in json format in this schema:
{Paper.model_json_schema()}
"""


In [26]:
for assistant in client.beta.assistants.list():
    print(assistant.id)


asst_ruWqBYrKwmsNT2igLfaZnjFR


In [ ]:
# Create research assistant
assistant = client.beta.assistants.create(
    instructions="You are a research assistant. Please extract the following information from the provided PDF and return json.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
)

In [27]:
assistant_id = "asst_ruWqBYrKwmsNT2igLfaZnjFR"


In [36]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Extract as much information as you can from this PDF.",
            "attachments": [{"file_id": file_id, "tools": [{"type": "file_search"}]}],
        }
    ],
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant_id, instructions=instructions
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
message_content = messages[0].content[0].text
print(message_content.value)

The extracted information from the PDF is as follows:

```json
{
  "title": "Carbon persistence of soils with long-term biosolids amendments in California agroecosystems",
  "authors": "Yocelyn B. Villa, Eddie Phosalath, Rebecca Ryals",
  "year": 2024,
  "doi": "10.1002/saj2.20632",
  "sites": [
    {
      "name": "Sacramento",
      "location": "irrigated, grazed grassland",
      "latitude": null,
      "longitude": null,
      "measurements": [
        {
          "group": "treatment",
          "group_details": "biosolids amended",
          "measurement_name": "Carbon and Nitrogen content",
          "measurement_year": "2024",
          "measurement_month": null,
          "measurement_depth": "0–10 cm, 30–50 cm, 75–100 cm",
          "soil_organic_carbon_g_per_kg": null,
          "details": "Higher carbon and nitrogen content in the free- and occluded-light fractions",
          "source_page": 370
        }
      ]
    },
    {
      "name": "Solano",
      "location": "rainfe

In [37]:
for i in range(3):
    print(df.loc[0].T)
    print()


Paper_Title                                       Carbon persistence of soils with long-term bio...
Authors                                            Yocelyn B. Villa, Eddie Phosalath, Rebecca Ryals
Site_Location                                                                     Sacramento County
Longitude                                                                             121°10′06.5″W
Latitude                                                                               38°20′06.3″N
Elevation_m                                                                                     NaN
Mean_precipitation                                                                              NaN
Min_Temp                                                                                        NaN
Mean_temp                                                                                       NaN
Max_temp                                                                                        NaN
